In [ ]:
from ultralytics import YOLO
import numpy as np


class YOLOSegmentation:
    def __init__(self, model_path):
        self.model = YOLO(model_path)

    def detect(self, img):
        height, width, channels = img.shape

        results = self.model.predict(source=img.copy(), save=False, save_txt=False)
        result = results[0]
        segmentation_contours_idx = []
        for seg in result.masks.segments:
            seg[:, 0] *= width
            seg[:, 1] *= height
            segment = np.array(seg, dtype=np.int32)
            segmentation_contours_idx.append(segment)

        bboxes = np.array(result.boxes.xyxy.cpu(), dtype="int")
        class_ids = np.array(result.boxes.cls.cpu(), dtype="int")
        scores = np.array(result.boxes.conf.cpu(), dtype="float").round(2)
        return bboxes, class_ids, segmentation_contours_idx, scores


In [ ]:
import cv2


img = cv2.imread("Path to image")
img = cv2.resize(img, None, fx=0.5, fy=0.5)


ys = YOLOSegmentation("yolov8m-seg.pt")

bboxes, classes, segmentations, scores = ys.detect(img)

#To draw contours, bbox and class of image uncomment it
# for bbox, class_id, seg, score in zip(bboxes, classes, segmentations, scores):
#     (x, y, x2, y2) = bbox
#     # if class_id == 29:
#     # cv2.rectangle(img, (x, y), (x2, y2), (255, 0, 0), 2)

#     cv2.polylines(img, [seg], True, (0, 0, 255), 1)

    # cv2.putText(
    #     img, str(class_id), (x, y - 10),
    #     cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255), 2)

# cv2.imshow("image", img)
# cv2.waitKey(0)

In [ ]:
# Все за пределами сегментированного изображения станет черным
points = np.array(segmentations)

mask = np.zeros(img.shape[:2], dtype=np.uint8)
cv2.drawContours(mask, [points], -1, (255, 255, 255), -1, cv2.LINE_AA)
cropped_img = cv2.bitwise_and(img, img, mask=mask)

# Display cropped image if you want to
# cv2.imshow("Cropped Image", cropped_img)
# cv2.waitKey(0)
cv2.imwrite(('Where to store cropped image'), cropped_img)

In [ ]:
image3 = cv2.imread(cropped_img) 
blurred_image = cv2.GaussianBlur(image3.copy(),(5,5),0)

edges = cv2.Canny(blurred_image, 100, 160)

plt.figure(figsize=[10,10])
cv2.imwrite(('Where to store contours'), edges)
plt.imshow(edges,cmap='Greys_r');plt.title("Edges Image");plt.axis("off")
